In [1]:
import numpy as np
import pandas as pd
import re

from matplotlib import pyplot as plt
from sklearn.metrics import r2_score

from datetime import timedelta,datetime

import sys
from path import Path

import plotly.graph_objects as go

In [2]:
class_path = Path("../../classes/pmmfs_ml")
sys.path.append(class_path)
from ml import ml

In [3]:
path_GR = "../Resources/raw/OxCGRT_latest(Aug31,2020).csv"   # Government Response
path_CD = "../Resources/raw/owid-covid-data(Aug31,2020).csv"   # COVID Data

In [4]:
data_GR = pd.read_csv(path_GR)    #Government Response
data_CD = pd.read_csv(path_CD)

/home/sami/anaconda3/envs/mlenv2/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# Drop rows from Government response for individual regions for CountryCodes USA and GBR

data_GR = data_GR[data_GR[["RegionName"]].isnull().values].copy()

In [6]:
df_GR = data_GR[['Date','CountryCode',]]
stringency_columns = [ x for x in data_GR if re.search("C\d|H1",x)]
income_support = [x for x in data_GR if re.search("E1",x)]

for x in stringency_columns + income_support:
    df_GR[x] = data_GR[x]
    
df_GR

/home/sami/anaconda3/envs/mlenv2/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Date,CountryCode,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,C3_Cancel public events,C3_Flag,C4_Restrictions on gatherings,C4_Flag,...,C5_Flag,C6_Stay at home requirements,C6_Flag,C7_Restrictions on internal movement,C7_Flag,C8_International travel controls,H1_Public information campaigns,H1_Flag,E1_Income support,E1_Flag
0,20200101,ABW,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,NaN
1,20200102,ABW,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,NaN
2,20200103,ABW,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,NaN
3,20200104,ABW,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,NaN
4,20200105,ABW,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58799,20200827,ZWE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58800,20200828,ZWE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58801,20200829,ZWE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58802,20200830,ZWE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_GR.isnull().sum()

Date                                        0
CountryCode                                 0
C1_School closing                        1166
C1_Flag                                 18029
C2_Workplace closing                     1268
C2_Flag                                 20349
C3_Cancel public events                  1204
C3_Flag                                 17224
C4_Restrictions on gatherings            1235
C4_Flag                                 19101
C5_Close public transport                1076
C5_Flag                                 28457
C6_Stay at home requirements             1237
C6_Flag                                 23086
C7_Restrictions on internal movement     1125
C7_Flag                                 23855
C8_International travel controls         1187
H1_Public information campaigns          1667
H1_Flag                                  9947
E1_Income support                        2154
E1_Flag                                 25658
dtype: int64

In [8]:
for x in df_GR.columns:
    if re.search("C\d|H1|E1",x):
        print(df_GR[x].value_counts())

3.0    20243
0.0    16863
2.0     5814
1.0     1054
Name: C1_School closing, dtype: int64
1.0    24094
0.0     3017
Name: C1_Flag, dtype: int64
0.0    19081
2.0    14554
3.0     6254
1.0     3983
Name: C2_Workplace closing, dtype: int64
1.0    19734
0.0     5057
Name: C2_Flag, dtype: int64
2.0    24004
0.0    16020
1.0     3912
Name: C3_Cancel public events, dtype: int64
1.0    25151
0.0     2765
Name: C3_Flag, dtype: int64
0.0    17866
4.0    12469
3.0     9677
2.0     3139
1.0      754
Name: C4_Restrictions on gatherings, dtype: int64
1.0    22667
0.0     3372
Name: C4_Flag, dtype: int64
0.0    27381
1.0     9272
2.0     7411
Name: C5_Close public transport, dtype: int64
1.0    12282
0.0     4401
Name: C5_Flag, dtype: int64
0.0    21849
2.0    10715
1.0     8905
3.0     2434
Name: C6_Stay at home requirements, dtype: int64
1.0    16532
0.0     5522
Name: C6_Flag, dtype: int64
0.0    22730
2.0    15807
1.0     5478
Name: C7_Restrictions on internal movement, dtype: int64
1.0    12912


In [9]:
df_GR = df_GR.fillna(0)

In [10]:
df_CD_1 = data_CD[["date","iso_code","population","population_density",'median_age',"total_cases","total_deaths"]]
df_CD_1 = df_CD_1.dropna()
df_CD_2 = data_CD[["date","iso_code","new_cases","new_deaths"]]
df_CD_2 = df_CD_2.fillna(0)

In [11]:
df_CD = df_CD_1.merge(df_CD_2,on=["date","iso_code"])

In [12]:
df_CD.head()

,date,iso_code,population,population_density,median_age,total_cases,total_deaths,new_cases,new_deaths
0,2020-03-13,ABW,106766.0,584.8,41.2,2.0,0.0,2.0,0.0
1,2020-03-20,ABW,106766.0,584.8,41.2,4.0,0.0,2.0,0.0
2,2020-03-24,ABW,106766.0,584.8,41.2,12.0,0.0,8.0,0.0
3,2020-03-25,ABW,106766.0,584.8,41.2,17.0,0.0,5.0,0.0
4,2020-03-26,ABW,106766.0,584.8,41.2,19.0,0.0,2.0,0.0


In [13]:
df_CD.shape

(35174, 9)

In [14]:
codes = df_CD["iso_code"].unique()

iso_dfs = []

total_cases_col = 2
tota_deaths_col = 3

for code in codes:
    iso_df = df_CD[df_CD["iso_code"] == code]
    if np.isnan(iso_df.iloc[0,total_cases_col]):
        iso_df.iloc[0,total_cases_col] = 0
    if np.isnan(iso_df.iloc[0,tota_deaths_col]):
        iso_df.iloc[0,tota_deaths_col] = 0
    
    iso_df = iso_df.fillna(method='ffill')
    
    if iso_df[["median_age"]].any().isnull().sum() > 0:
        print("Bad")
        
    iso_dfs.append(iso_df)

df_CD = pd.DataFrame()

for iso_df in iso_dfs:
    df_CD = df_CD.append(iso_df)
    
# df_CD = df_CD.fillna(method='ffill')    
df_CD.isnull().sum()

date                  0
iso_code              0
population            0
population_density    0
median_age            0
total_cases           0
total_deaths          0
new_cases             0
new_deaths            0
dtype: int64

In [15]:
df_CD

,date,iso_code,population,population_density,median_age,total_cases,total_deaths,new_cases,new_deaths
0,2020-03-13,ABW,1.067660e+05,584.800,41.2,2.0,0.0,2.0,0.0
1,2020-03-20,ABW,1.067660e+05,584.800,41.2,4.0,0.0,2.0,0.0
2,2020-03-24,ABW,1.067660e+05,584.800,41.2,12.0,0.0,8.0,0.0
3,2020-03-25,ABW,1.067660e+05,584.800,41.2,17.0,0.0,5.0,0.0
4,2020-03-26,ABW,1.067660e+05,584.800,41.2,19.0,0.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...
35169,2020-08-27,OWID_WRL,7.794799e+09,58.045,30.9,24201449.0,825936.0,276699.0,6297.0
35170,2020-08-28,OWID_WRL,7.794799e+09,58.045,30.9,24484672.0,832011.0,283223.0,6075.0
35171,2020-08-29,OWID_WRL,7.794799e+09,58.045,30.9,24761119.0,837466.0,276447.0,5455.0
35172,2020-08-30,OWID_WRL,7.794799e+09,58.045,30.9,25029850.0,843154.0,268731.0,5688.0


In [16]:
df_GR.columns

Index(['Date', 'CountryCode', 'C1_School closing', 'C1_Flag',
       'C2_Workplace closing', 'C2_Flag', 'C3_Cancel public events', 'C3_Flag',
       'C4_Restrictions on gatherings', 'C4_Flag', 'C5_Close public transport',
       'C5_Flag', 'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'H1_Public information campaigns',
       'H1_Flag', 'E1_Income support', 'E1_Flag'],
      dtype='object')

In [17]:
df_GR.isnull().sum()

Date                                    0
CountryCode                             0
C1_School closing                       0
C1_Flag                                 0
C2_Workplace closing                    0
C2_Flag                                 0
C3_Cancel public events                 0
C3_Flag                                 0
C4_Restrictions on gatherings           0
C4_Flag                                 0
C5_Close public transport               0
C5_Flag                                 0
C6_Stay at home requirements            0
C6_Flag                                 0
C7_Restrictions on internal movement    0
C7_Flag                                 0
C8_International travel controls        0
H1_Public information campaigns         0
H1_Flag                                 0
E1_Income support                       0
E1_Flag                                 0
dtype: int64

In [18]:
df_CD.columns

Index(['date', 'iso_code', 'population', 'population_density', 'median_age',
       'total_cases', 'total_deaths', 'new_cases', 'new_deaths'],
      dtype='object')

In [19]:
df_GR = df_GR.rename(columns={"Date":"date","CountryCode":"iso_code"})

In [20]:
def formatDate(dt):
    d_str = str(dt)
    year = d_str[0:4]
    month = d_str[4:6]
    day = d_str[6:8]
    dt_str = year + "-" + month + "-" + day
     
    return dt_str

In [21]:
type(df_GR["date"][0])

numpy.int64

In [22]:
formatDate(df_GR["date"][0])

'2020-01-01'

In [23]:
df_GR["date"] = df_GR["date"].apply(lambda x:formatDate(x))

In [24]:
df_GR.head()

,date,iso_code,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,C3_Cancel public events,C3_Flag,C4_Restrictions on gatherings,C4_Flag,...,C5_Flag,C6_Stay at home requirements,C6_Flag,C7_Restrictions on internal movement,C7_Flag,C8_International travel controls,H1_Public information campaigns,H1_Flag,E1_Income support,E1_Flag
0,2020-01-01,ABW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-01-02,ABW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-01-03,ABW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-01-04,ABW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-01-05,ABW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
df_GR.shape

(45140, 21)

In [26]:
df_CD.head()

,date,iso_code,population,population_density,median_age,total_cases,total_deaths,new_cases,new_deaths
0,2020-03-13,ABW,106766.0,584.8,41.2,2.0,0.0,2.0,0.0
1,2020-03-20,ABW,106766.0,584.8,41.2,4.0,0.0,2.0,0.0
2,2020-03-24,ABW,106766.0,584.8,41.2,12.0,0.0,8.0,0.0
3,2020-03-25,ABW,106766.0,584.8,41.2,17.0,0.0,5.0,0.0
4,2020-03-26,ABW,106766.0,584.8,41.2,19.0,0.0,2.0,0.0


In [27]:
final_df = df_GR.merge(df_CD,on=["date","iso_code"])
final_df

,date,iso_code,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,C3_Cancel public events,C3_Flag,C4_Restrictions on gatherings,C4_Flag,...,H1_Flag,E1_Income support,E1_Flag,population,population_density,median_age,total_cases,total_deaths,new_cases,new_deaths
0,2020-03-13,ABW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,106766.0,584.800,41.2,2.0,0.0,2.0,0.0
1,2020-03-20,ABW,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,106766.0,584.800,41.2,4.0,0.0,2.0,0.0
2,2020-03-24,ABW,3.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,106766.0,584.800,41.2,12.0,0.0,8.0,0.0
3,2020-03-25,ABW,3.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,106766.0,584.800,41.2,17.0,0.0,5.0,0.0
4,2020-03-26,ABW,3.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,106766.0,584.800,41.2,19.0,0.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31800,2020-08-27,ZWE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,14862927.0,42.729,19.6,6251.0,179.0,55.0,13.0
31801,2020-08-28,ZWE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,14862927.0,42.729,19.6,6292.0,189.0,41.0,10.0
31802,2020-08-29,ZWE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,14862927.0,42.729,19.6,6388.0,195.0,96.0,6.0
31803,2020-08-30,ZWE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,14862927.0,42.729,19.6,6406.0,196.0,18.0,1.0


In [28]:
df1 = final_df[['date','iso_code','population','population_density','median_age','C6_Stay at home requirements','C7_Restrictions on internal movement','C8_International travel controls','total_cases','total_deaths']]
df2 = final_df[['date','iso_code','population','population_density','median_age','C1_School closing','C2_Workplace closing','C3_Cancel public events','C6_Stay at home requirements','C7_Restrictions on internal movement','C8_International travel controls','total_cases','total_deaths']]
df3 = final_df[['date','iso_code','population','population_density','median_age','C1_School closing','C2_Workplace closing','C3_Cancel public events','C4_Restrictions on gatherings','C5_Close public transport','C6_Stay at home requirements','C7_Restrictions on internal movement','C8_International travel controls','total_cases','total_deaths']]
df4 = final_df[['date','iso_code','population','population_density','median_age','C6_Stay at home requirements','C7_Restrictions on internal movement','C8_International travel controls','E1_Income support','total_cases','total_deaths']]


dfs = [df1, df2, df3, df4]


In [29]:
def get_iso_dicts(df):
    
    iso_codes = df["iso_code"].unique()
    
    iso_dicts = {}
    
    for code in iso_codes:
        
        iso_dicts[code] = {}
        
        for day in future:
            
            date_col = "_".join(["date",day])
            
            df_iso = final_df[final_df["iso_code"]==code][["date","iso_code"]]
            
            df_iso[date_col] = pd.to_datetime(df_iso["date"]) + timedelta(days=int(day))
            
            df_iso[date_col] = df_iso[date_col].astype(str)

            iso_dicts[code][day] = df_iso
            
            
    return iso_dicts, iso_codes

In [30]:
def get_cum_days(*iso_codes, **iso_dicts):
    
    days = []

    for day in future:
        
        cum_df = pd.DataFrame()
        
        for code in iso_codes:
            
            cum_df = cum_df.append(iso_dicts[code][day])


        days.append(cum_df.reset_index(drop=True))
    
    return days

In [31]:
def get_amended_days(*days):
    
    m_days = []
    
    for i,day in enumerate(future):
        
        m_days.append(days[i].merge(final_df,left_on=["date_" + day,"iso_code"],right_on=["date","iso_code"],how="left",suffixes=["","_" + day]))
        
        m_days[i] = m_days[i].reset_index(drop=True)

        m_days[i] = m_days[i][["date","iso_code","date_" + day,"total_cases","total_deaths"]]
        
        m_days[i].rename(columns={"total_cases":"total_cases_" + day,
                           "total_deaths":"total_deaths_" + day},inplace=True)
        
    return m_days

In [32]:
future = ["30","45","60","75"]

def get_days(df):

    iso_dicts, iso_codes = get_iso_dicts(df)
    
    days = get_cum_days(*iso_codes, **iso_dicts)
        
    days = get_amended_days(*days)

    return days

In [33]:
layers = { 
            "number_input_features": 0,   # number of input features will vary
            "n_layers": 4, 
            "l0":{"number_hidden_nodes":18,"activation_function":"relu"},
            "l1":{"number_hidden_nodes":9,"activation_function":"relu"},
            "l2":{"number_hidden_nodes":4,"activation_function":"relu"},
            "l3":{"number_hidden_nodes":1,"activation_function":"linear"}
        }

In [34]:
# Regression Analysis

models = []

for n,df in enumerate(dfs):

    features = df.columns
    
    days = get_days(df)
        
    for target in ["total_cases","total_deaths"]:

        print(f"{target} %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%>\n\n")
        for i,day in enumerate(future):
            
            print(f"\n\n>>>>>>>> {day} >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ")

            df_n = df.merge(days[i],on=["date","iso_code"],how="left")
            print(df_n.columns)

            features = [f for f in features if re.search("_\d{2}$",f) == None]
            features = features[2:]  # Remove date and iso_code

            layers["number_input_features"] = len(features)

            md = ml(df=df_n,
                    feature_set=n,
                    test_size=0.05,
                    title=f"{target} - {day} days out",
                    target=target,
                    period=day,
                    **layers
                   )

            md.train_test(epochs=100)

            models.append(md)
            

total_cases %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%>




>>>>>>>> 30 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 
Index(['date', 'iso_code', 'population', 'population_density', 'median_age',
       'C6_Stay at home requirements', 'C7_Restrictions on internal movement',
       'C8_International travel controls', 'total_cases', 'total_deaths',
       'date_30', 'date_30', 'total_cases_30', 'total_deaths_30'],
      dtype='object')


 Training and testing - 30 days ahead


target_n = total_cases_30

features = ['population', 'population_density', 'median_age', 'C6_Stay at home requirements', 'C7_Restrictions on internal movement', 'C8_International travel controls', 'total_cases', 'total_deaths']
population

population_density

median_age

C6_Stay at home requirements

C7_Restrictions on internal movement

C8_International travel controls

total_cases

total_deaths

Model: "sequential"
_________________________________________________________________
Layer (

793/793 [==============================] - 2s 3ms/step - loss: 0.0029 - mse: 0.0029
Epoch 68/100
793/793 [==============================] - 2s 3ms/step - loss: 0.0030 - mse: 0.0030
Epoch 69/100
793/793 [==============================] - 2s 2ms/step - loss: 0.0028 - mse: 0.0028
Epoch 70/100
793/793 [==============================] - 2s 2ms/step - loss: 0.0029 - mse: 0.0029
Epoch 71/100
793/793 [==============================] - 2s 3ms/step - loss: 0.0034 - mse: 0.0034
Epoch 72/100
793/793 [==============================] - 2s 2ms/step - loss: 0.0029 - mse: 0.0029
Epoch 73/100
793/793 [==============================] - 2s 3ms/step - loss: 0.0030 - mse: 0.0030
Epoch 74/100
793/793 [==============================] - 2s 2ms/step - loss: 0.0030 - mse: 0.0030
Epoch 75/100
793/793 [==============================] - 2s 3ms/step - loss: 0.0028 - mse: 0.0028
Epoch 76/100
793/793 [==============================] - 2s 3ms/step - loss: 0.0030 - mse: 0.0030
Epoch 77/100
793/793 [=====================

Epoch 33/100
721/721 [==============================] - 2s 3ms/step - loss: 0.0063 - mse: 0.0063
Epoch 34/100
721/721 [==============================] - 2s 3ms/step - loss: 0.0065 - mse: 0.0065
Epoch 35/100
721/721 [==============================] - 2s 3ms/step - loss: 0.0067 - mse: 0.0067
Epoch 36/100
721/721 [==============================] - 2s 3ms/step - loss: 0.0060 - mse: 0.0060
Epoch 37/100
721/721 [==============================] - 2s 3ms/step - loss: 0.0064 - mse: 0.0064
Epoch 38/100
721/721 [==============================] - 2s 3ms/step - loss: 0.0066 - mse: 0.0066
Epoch 39/100
721/721 [==============================] - 2s 3ms/step - loss: 0.0057 - mse: 0.0057
Epoch 40/100
721/721 [==============================] - 2s 3ms/step - loss: 0.0064 - mse: 0.0064
Epoch 41/100
721/721 [==============================] - 2s 3ms/step - loss: 0.0064 - mse: 0.0064
Epoch 42/100
721/721 [==============================] - 2s 3ms/step - loss: 0.0064 - mse: 0.0064
Epoch 43/100
721/721 [========

Epoch 1/100
648/648 [==============================] - 2s 3ms/step - loss: 0.4159 - mse: 0.4159
Epoch 2/100
648/648 [==============================] - 2s 3ms/step - loss: 0.0847 - mse: 0.0847
Epoch 3/100
648/648 [==============================] - 2s 3ms/step - loss: 0.0612 - mse: 0.0612
Epoch 4/100
648/648 [==============================] - 2s 3ms/step - loss: 0.0479 - mse: 0.0479
Epoch 5/100
648/648 [==============================] - 2s 3ms/step - loss: 0.0411 - mse: 0.0411
Epoch 6/100
648/648 [==============================] - 2s 3ms/step - loss: 0.0339 - mse: 0.0339
Epoch 7/100
648/648 [==============================] - 2s 3ms/step - loss: 0.0313 - mse: 0.0313
Epoch 8/100
648/648 [==============================] - 2s 3ms/step - loss: 0.0283 - mse: 0.0283
Epoch 9/100
648/648 [==============================] - 2s 3ms/step - loss: 0.0274 - mse: 0.0274
Epoch 10/100
648/648 [==============================] - 2s 3ms/step - loss: 0.0256 - mse: 0.0256
Epoch 11/100
648/648 [=================

648/648 [==============================] - 2s 3ms/step - loss: 0.0084 - mse: 0.0084
Epoch 86/100
648/648 [==============================] - 2s 2ms/step - loss: 0.0084 - mse: 0.0084
Epoch 87/100
648/648 [==============================] - 2s 3ms/step - loss: 0.0081 - mse: 0.0081
Epoch 88/100
648/648 [==============================] - 2s 3ms/step - loss: 0.0083 - mse: 0.0083
Epoch 89/100
648/648 [==============================] - 2s 3ms/step - loss: 0.0081 - mse: 0.0081
Epoch 90/100
648/648 [==============================] - 2s 2ms/step - loss: 0.0089 - mse: 0.0089
Epoch 91/100
648/648 [==============================] - 2s 2ms/step - loss: 0.0079 - mse: 0.0079
Epoch 92/100
648/648 [==============================] - 2s 3ms/step - loss: 0.0085 - mse: 0.0085
Epoch 93/100
648/648 [==============================] - 2s 3ms/step - loss: 0.0075 - mse: 0.0075
Epoch 94/100
648/648 [==============================] - 2s 2ms/step - loss: 0.0087 - mse: 0.0087
Epoch 95/100
648/648 [=====================

...

Epoch 1/100
721/721 [==============================] - 3s 4ms/step - loss: 0.2164 - mse: 0.2164
Epoch 2/100
721/721 [==============================] - 3s 5ms/step - loss: 0.1053 - mse: 0.1053
Epoch 3/100
721/721 [==============================] - 4s 5ms/step - loss: 0.0824 - mse: 0.0824
Epoch 4/100
721/721 [==============================] - 3s 4ms/step - loss: 0.0642 - mse: 0.0642
Epoch 5/100
721/721 [==============================] - 3s 5ms/step - loss: 0.0533 - mse: 0.0533
Epoch 6/100
721/721 [==============================] - 3s 4ms/step - loss: 0.0450 - mse: 0.0450
Epoch 7/100
721/721 [==============================] - 3s 4ms/step - loss: 0.0399 - mse: 0.0399
Epoch 8/100
721/721 [==============================] - 2s 3ms/step - loss: 0.0368 - mse: 0.0368
Epoch 9/100
721/721 [==============================] - 3s 4ms/step - loss: 0.0339 - mse: 0.0339
Epoch 10/100
721/721 [==============================] - 2s 3ms/step - loss: 0.0321 - mse: 0.0321
Epoch 11/100
721/721 [=================

721/721 [==============================] - 3s 4ms/step - loss: 0.0098 - mse: 0.0098
Epoch 86/100
721/721 [==============================] - 3s 4ms/step - loss: 0.0106 - mse: 0.0106
Epoch 87/100
721/721 [==============================] - 3s 4ms/step - loss: 0.0095 - mse: 0.0095
Epoch 88/100
721/721 [==============================] - 3s 4ms/step - loss: 0.0101 - mse: 0.0101
Epoch 89/100
721/721 [==============================] - 3s 5ms/step - loss: 0.0095 - mse: 0.0095
Epoch 90/100
721/721 [==============================] - 3s 4ms/step - loss: 0.0101 - mse: 0.0101
Epoch 91/100
721/721 [==============================] - 3s 4ms/step - loss: 0.0097 - mse: 0.0097
Epoch 92/100
721/721 [==============================] - 3s 4ms/step - loss: 0.0113 - mse: 0.0113
Epoch 93/100
721/721 [==============================] - 3s 5ms/step - loss: 0.0096 - mse: 0.0096
Epoch 94/100
721/721 [==============================] - 3s 4ms/step - loss: 0.0104 - mse: 0.0104
Epoch 95/100
721/721 [=====================

648/648 [==============================] - 3s 5ms/step - loss: 0.0182 - mse: 0.0182
Epoch 51/100
648/648 [==============================] - 3s 5ms/step - loss: 0.0190 - mse: 0.0190
Epoch 52/100
648/648 [==============================] - 2s 4ms/step - loss: 0.0190 - mse: 0.0190
Epoch 53/100
648/648 [==============================] - 2s 3ms/step - loss: 0.0173 - mse: 0.0173
Epoch 54/100
648/648 [==============================] - 2s 4ms/step - loss: 0.0256 - mse: 0.0256
Epoch 55/100
648/648 [==============================] - 3s 4ms/step - loss: 0.0168 - mse: 0.0168
Epoch 56/100
648/648 [==============================] - 2s 3ms/step - loss: 0.0181 - mse: 0.0181
Epoch 57/100
648/648 [==============================] - 2s 4ms/step - loss: 0.0168 - mse: 0.0168
Epoch 58/100
648/648 [==============================] - 2s 3ms/step - loss: 0.0168 - mse: 0.0168
Epoch 59/100
648/648 [==============================] - 3s 4ms/step - loss: 0.0172 - mse: 0.0172
Epoch 60/100
648/648 [=====================

582/582 [==============================] - 2s 4ms/step - loss: 0.0452 - mse: 0.0452
Epoch 16/100
582/582 [==============================] - 3s 5ms/step - loss: 0.0441 - mse: 0.0441
Epoch 17/100
582/582 [==============================] - 2s 4ms/step - loss: 0.0524 - mse: 0.0524
Epoch 18/100
582/582 [==============================] - 2s 3ms/step - loss: 0.0388 - mse: 0.0388
Epoch 19/100
582/582 [==============================] - 2s 4ms/step - loss: 0.0407 - mse: 0.0407
Epoch 20/100
582/582 [==============================] - 2s 4ms/step - loss: 0.0403 - mse: 0.0403
Epoch 21/100
582/582 [==============================] - 3s 5ms/step - loss: 0.0350 - mse: 0.0350
Epoch 22/100
582/582 [==============================] - 3s 5ms/step - loss: 0.0340 - mse: 0.0340
Epoch 23/100
582/582 [==============================] - 2s 4ms/step - loss: 0.0347 - mse: 0.0347
Epoch 24/100
582/582 [==============================] - 2s 4ms/step - loss: 0.0321 - mse: 0.0321
Epoch 25/100
582/582 [=====================

582/582 [==============================] - 2s 4ms/step - loss: 0.0203 - mse: 0.0203
Epoch 100/100
582/582 [==============================] - 2s 3ms/step - loss: 0.0197 - mse: 0.0197
Training r2_score = 0.9814214979533804
Testing  r2_score = 0.9874140895521862


In [39]:
for md in models:
    df = md.get_df()
    model = md.get_model()
    features = md.get_features()
    
    target = md.get_target() + "_" + md.get_period()
    
    X = df[features]
    
    X_scaler = md.get_X_scaler()
    X_scaled = X_scaler.transform(X)
    y_pred_scaled = model.predict(X_scaled)
    
    x_vals = list(df["date"])
    y_test_scaled = md.get_y_scaler().transform(df[target].values.reshape(-1,1)).tolist()
    y_test = [y[0] for y in y_test_scaled]
    y_pred_scaled = model.predict(X_scaled).tolist()
    y_pred = [y[0] for y in y_pred_scaled]
    
    df_c = pd.DataFrame({"X":x_vals,"y_test":y_test, "y_pred": y_pred})
    
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_c["X"], y=df_c["y_test"],mode="lines",name="Actual"))
    fig.add_trace(go.Scatter(x=df_c["X"], y=df_c["y_pred"],mode="lines",name="Predicted"))
    fig.update_layout(title=md.get_title())
    
    features = [f[:2] if re.search("^[CE]\d",f) else f for f in features]
    
    fig.write_image(f"graphs/target - {md.get_target()} - features - {features} - period - {md.get_period()}.png")
    

In [47]:
final_df.tail()

,date,iso_code,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,C3_Cancel public events,C3_Flag,C4_Restrictions on gatherings,C4_Flag,...,H1_Flag,E1_Income support,E1_Flag,population,population_density,median_age,total_cases,total_deaths,new_cases,new_deaths
31800,2020-08-27,ZWE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,14862927.0,42.729,19.6,6251.0,179.0,55.0,13.0
31801,2020-08-28,ZWE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,14862927.0,42.729,19.6,6292.0,189.0,41.0,10.0
31802,2020-08-29,ZWE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,14862927.0,42.729,19.6,6388.0,195.0,96.0,6.0
31803,2020-08-30,ZWE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,14862927.0,42.729,19.6,6406.0,196.0,18.0,1.0
31804,2020-08-31,ZWE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,14862927.0,42.729,19.6,6412.0,196.0,6.0,0.0


In [45]:
days = get_days(final_df)

In [46]:
days[0]

,date,iso_code,date_30,date_30,total_cases_30,total_deaths_30
0,2020-03-13,ABW,2020-04-12,2020-04-12,92.0,0.0
1,2020-03-20,ABW,2020-04-19,2020-04-19,96.0,2.0
2,2020-03-24,ABW,2020-04-23,2020-04-23,100.0,2.0
3,2020-03-25,ABW,2020-04-24,2020-04-24,100.0,2.0
4,2020-03-26,ABW,2020-04-25,2020-04-25,100.0,2.0
...,...,...,...,...,...,...
31800,2020-08-27,ZWE,2020-09-26,NaN,NaN,NaN
31801,2020-08-28,ZWE,2020-09-27,NaN,NaN,NaN
31802,2020-08-29,ZWE,2020-09-28,NaN,NaN,NaN
31803,2020-08-30,ZWE,2020-09-29,NaN,NaN,NaN


Use the days list and the different models to predict cases as far out as possible

In [54]:
# predict_df
codes = final_df["iso_code"].unique()

for code in codes:
    iso_df = final_df[final_df["iso_code"] == code]
    print(iso_df["date"][-75:])
    
    



87     2020-06-18
88     2020-06-19
89     2020-06-20
90     2020-06-21
91     2020-06-22
          ...    
157    2020-08-27
158    2020-08-28
159    2020-08-29
160    2020-08-30
161    2020-08-31
Name: date, Length: 75, dtype: object
321    2020-06-18
322    2020-06-19
323    2020-06-20
324    2020-06-21
325    2020-06-22
          ...    
391    2020-08-27
392    2020-08-28
393    2020-08-29
394    2020-08-30
395    2020-08-31
Name: date, Length: 75, dtype: object
484    2020-06-18
485    2020-06-19
486    2020-06-20
487    2020-06-21
488    2020-06-22
          ...    
554    2020-08-27
555    2020-08-28
556    2020-08-29
557    2020-08-30
558    2020-08-31
Name: date, Length: 75, dtype: object
660    2020-06-18
661    2020-06-19
662    2020-06-20
663    2020-06-21
664    2020-06-22
          ...    
730    2020-08-27
731    2020-08-28
732    2020-08-29
733    2020-08-30
734    2020-08-31
Name: date, Length: 75, dtype: object
898    2020-06-18
899    2020-06-19
900    2020-06-20
90

15908    2020-06-18
15909    2020-06-19
15910    2020-06-20
15911    2020-06-21
15912    2020-06-22
            ...    
15978    2020-08-27
15979    2020-08-28
15980    2020-08-29
15981    2020-08-30
15982    2020-08-31
Name: date, Length: 75, dtype: object
16078    2020-06-18
16079    2020-06-19
16080    2020-06-20
16081    2020-06-21
16082    2020-06-22
            ...    
16148    2020-08-27
16149    2020-08-28
16150    2020-08-29
16151    2020-08-30
16152    2020-08-31
Name: date, Length: 75, dtype: object
16249    2020-06-18
16250    2020-06-19
16251    2020-06-20
16252    2020-06-21
16253    2020-06-22
            ...    
16319    2020-08-27
16320    2020-08-28
16321    2020-08-29
16322    2020-08-30
16323    2020-08-31
Name: date, Length: 75, dtype: object
16415    2020-06-18
16416    2020-06-19
16417    2020-06-20
16418    2020-06-21
16419    2020-06-22
            ...    
16485    2020-08-27
16486    2020-08-28
16487    2020-08-29
16488    2020-08-30
16489    2020-08-31
Name: 